In [79]:
# input 1 (InputLayer) (25, 25, 30, 1) 0
# conv3d 1 (Conv3D) (23, 23, 24, 8) 512
# conv3d 2 (Conv3D) (21, 21, 20, 16) 5776
# conv3d 3 (Conv3D) (19, 19, 18, 32) 13856
# reshape 1 (Reshape) (19, 19, 576) 0
# conv2d 1 (Conv2D) (17, 17, 64) 331840
# flatten 1 (Flatten) (18496) 0
# dense 1 (Dense) (256) 4735232
# dropout 1 (Dropout) (256) 0
# dense 2 (Dense) (128) 32896
# dropout 2 (Dropout) (128) 0
# dense 3 (Dense) (16) 2064

In [287]:
from hybridsn import HybridSN
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
import torch

In [288]:
input = torch.randn(1, 1, 25, 25, 30)

In [289]:
#summary(HybridSN(), input)

In [290]:
HybridSN()

HybridSN(
  (conv3d): Sequential(
    (0): Conv3d(1, 8, kernel_size=(3, 3, 7), stride=(1, 1, 1))
    (1): ReLU()
    (2): Conv3d(8, 16, kernel_size=(3, 3, 5), stride=(1, 1, 1))
    (3): ReLU()
    (4): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (5): ReLU()
  )
  (conv2d): Sequential(
    (0): Conv2d(30, 16, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU()
  )
  (linear): Sequential(
    (0): Linear(in_features=2560, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
  )
)

In [291]:
conv3d = nn.Sequential(
            nn.Conv3d(1, 8, (3, 3, 7)),
            nn.ReLU(),
            nn.Conv3d(8, 16, (3, 3, 5)),
            nn.ReLU(),
            nn.Conv3d(16, 32, (3, 3, 3)),
            nn.ReLU()
        )
x = conv3d(input)

In [292]:
x.shape

torch.Size([1, 32, 19, 19, 18])

In [293]:
y = x.reshape((-1, x.shape[1]* x.shape[4],x.shape[2] , x.shape[3]))
y.shape

torch.Size([1, 576, 19, 19])

In [294]:
conv2d = nn.Sequential(
            nn.Conv2d(576, 64, (3, 3)),
            nn.ReLU()
        )


In [295]:
y1 = conv2d(y)
y1.shape

torch.Size([1, 64, 17, 17])

In [296]:
linear = nn.Sequential(
            nn.Linear(1156,16),
            nn.ReLU(),
            nn.Dropout(p=0.4),
            nn.Linear(16, 1),
            nn.Dropout(p=0.4),
            nn.ReLU()
        )

In [297]:
y2 = torch.flatten(y1)
y2.shape

torch.Size([18496])

In [298]:
y1.shape

torch.Size([1, 64, 17, 17])

In [299]:
y4 = y2.view(-1, 16)

print(y4.shape)
y5 = y4.reshape(y4.shape[1],y4.shape[0])
y5.shape

torch.Size([1156, 16])


torch.Size([16, 1156])

In [300]:
y3 = linear(y5)
y3.shape

torch.Size([16, 1])

In [301]:
y7 = y3.view(-1, 16)
y7.shape

torch.Size([1, 16])

In [305]:
y7.shape

torch.Size([1, 16])

In [271]:
y3.shape

torch.Size([16, 1])

In [303]:
#output with shape 1,16
y7

tensor([[0.0349, 0.0523, 0.0667, 0.0000, 0.0542, 0.0626, 0.0344, 0.0579, 0.0214,
         0.0695, 0.0542, 0.0000, 0.0664, 0.0549, 0.0405, 0.0515]],
       grad_fn=<ViewBackward>)

In [304]:
#output with shape 16,1
y3

tensor([[0.0349],
        [0.0523],
        [0.0667],
        [0.0000],
        [0.0542],
        [0.0626],
        [0.0344],
        [0.0579],
        [0.0214],
        [0.0695],
        [0.0542],
        [0.0000],
        [0.0664],
        [0.0549],
        [0.0405],
        [0.0515]], grad_fn=<ReluBackward0>)